# Extraction of temperature data out of the Climate12K LiPD files

This notebook extracts the temperature data of all temperature series in the Climate12K database and exports them in a tab-separated file. To get the latest version of the data, just click on *Cell* &rarr; *Run all*. When the notebook is finished (i.e. if you see a table [at the bottom](#final)), you can download the temperature data [here](data/binned-temperature-data.tsv) and the corresponding meta data [here](data/meta.tsv).

This notebook downloads the latest version of the QC sheet from [google docs](https://docs.google.com/spreadsheets/d/1JEm791Nhd4fUuyqece51CSlbR2A2I-pf8B0kFgwynug/edit#gid=238421392) and the corresponding LiPD files. It has been developed by Philipp Sommer (philipp.sommer@unil.ch), please do not hesitate to get in touch if you run into any problems.

**Things you might want to adapt:**

- the `binwidth` (by default 100, see [here](#binwidth)). We have to make averages over the time series in order to put everything together
- the records you want to download (see [here](#filter))
- the meta data for the temperature export (see [here](#meta-cols))

In [1]:
import pandas as pd
import lipd
import numpy as np
import contextlib
import os
import os.path as osp
from urllib import request

<a id=binwidth></a>We have to specify the bin widths in years. Each timeseries will be averaged into bins with this length in order to merge them all together.

In [2]:
binwidth = 100

Read in the QC spreadsheet from https://docs.google.com/spreadsheets/d/1JEm791Nhd4fUuyqece51CSlbR2A2I-pf8B0kFgwynug/

In [3]:
uri = 'https://docs.google.com/spreadsheets/d/1JEm791Nhd4fUuyqece51CSlbR2A2I-pf8B0kFgwynug/export?format=tsv&gid=1671690413'

df = pd.read_csv(uri, '\t', index_col='dataSetName')
df

,DK's(keep),TSid,createdBy,siteName,countryOcean,lat,lon,elevation,pub1_doi,pub2_doi,...,maxYear,medianHoloceneResolution,agesPerKyr,inCompilation,useInGlobalHolocene,QC Certification,QC comments,link to lipdverse,Remaining issues,last updated
dataSetName,,,,,,,,,,,,,,,,,,,,,
RR1313_23PC,2079.0,RNAQJ20Sh7j,PalMod2LiPD,RR1313_23PC,NaN,-4.4939,145.6703,-712.0,10.1029/2018PA003455,NaN,...,1.618600e+07,NaN,-0.007098,Temp12k,NaN,LJ,NaN,http://lipdverse.org/globalHolocene/current_ve...,NaN,v29
NevadoHuascaran.Thompson.1995,0.4,IC95THHU02D,matlab,Nevado Huascaran Core 2,NaN,-9.0000,-77.5000,6048.0,10.1126/science.269.5220.46,NaN,...,1.909900e+04,NaN,0.000000,Temp12k,FALSE,TO,100 year mean values,http://lipdverse.org/globalHolocene/current_ve...,NaN,before July 15
165_1002C,4.0,R7iVi9SqggU,PalMod2LiPD,165_1002C,Atlantic; Caribbean Sea,10.7061,-65.1694,-892.6,10.2973/odp.proc.sr.165.030.2000,10.2973/odp.proc.sr.165.017.2000,...,1.649700e+05,450.0000,0.000000,Temp12k,TRUE,"DSK,PMS, AdV",fails resolution; But chronology is good and i...,http://lipdverse.org/globalHolocene/current_ve...,NaN,before July 15
18287-3.Kienast.2001,5.0,GH7ecfa0de,matlab,18287-3,Pacific; South China Sea,5.7000,110.7000,-598.0,10.1126/science.1057131,NaN,...,1.488500e+04,207.0000,0.000000,Temp12k,TRUE,PMS,firs two dates are 3800years apart whereas the...,http://lipdverse.org/globalHolocene/current_ve...,NaN,before July 15
2005-804.Ledu.2013,8.0,PYT93BAHDDE,matlab,2005-804,Arctic; Northwestern Passages,68.9925,-106.5735,-118.0,10.1016/j.quascirev.2010.06.018,10.5194/cp-10-1605-2014,...,7.730800e+03,168.1000,0.000000,Temp12k,TRUE,PMS,it seems that the annual temp are calculated b...,http://lipdverse.org/globalHolocene/current_ve...,The name of the site should be LS009 according...,before July 15
2005-804.Ledu.2013,9.0,PYTGVVDEA4T,matlab,2005-804,Arctic; Northwestern Passages,68.9925,-106.5735,-118.0,10.1016/j.quascirev.2010.06.018,10.5194/cp-10-1605-2014,...,7.730800e+03,168.1000,0.000000,Temp12k,FALSE,PMS,there is something wrong in this Lipd file it ...,http://lipdverse.org/globalHolocene/current_ve...,there is something wrong in this Lipd file it ...,before July 15
3MPond.Pellatt.2000,10.0,Rlf3YywN7Sz,holoXL2lipd,3M Pond,North America>Canada>British Columbia,49.9833,-121.2167,1950.0,10.1080/15230430.2000.12003341,NaN,...,1.071428e+04,305.1945,0.000000,Temp12k,TRUE,MT,"MT: Digitized data (with GraphGrabber), the DO...",http://lipdverse.org/globalHolocene/current_ve...,NaN,before July 15
74KL_TEX86.Huguet.2006,11.0,GH77052c58,matlab,74KL,Indian; Arabian Sea,14.3000,57.3000,-3212.0,10.1029/2005PA001215,NaN,...,1.480000e+04,330.0000,0.000000,Temp12k,TRUE,PMS,I cannot find this data in Pangea/NOAA only th...,http://lipdverse.org/globalHolocene/current_ve...,cannot find URL for data,before July 15
74KL_UK37.Huguet.2006,12.0,GH4c7e3422,matlab,74KL,Indian; Arabian Sea,14.3000,57.3000,-3212.0,10.1029/2005PA001215,NaN,...,1.480000e+04,330.0000,0.000000,Temp12k,TRUE,PMS,I cannot find this data in Pangea/NOAA only th...,http://lipdverse.org/globalHolocene/current_ve...,cannot find URL for data,before July 15


<a id='filter'></a>Filter the data to select a subset of the entire dataset

In [17]:
df = df[(df.inCompilation == 'Temp12k') & (df.useInGlobalHolocene == "TRUE")]

Read in the LipD data from https://www.dropbox.com/sh/n2ck0y54b9es6lg/AADXVO0lf6Xb1Lem-b2iLVU4a?dl=0

In [ ]:
overwrite = False

In [5]:
%%time
if not osp.exists('data'):
    os.makedirs('data')
for name, html_uri in df['link to lipdverse'].dropna().drop_duplicates().items():
    uri = osp.splitext(html_uri)[0] + '.lpd'
    target = 'data/' + name + '.lpd'
    if overwrite or not osp.exists(target):
        print('downloading ' + uri)
        request.urlretrieve(uri,  target)

downloading http://lipdverse.org/globalHolocene/current_version/RR1313_23PC.lpd
downloading http://lipdverse.org/globalHolocene/current_version/NevadoHuascaran.Thompson.1995.lpd
downloading http://lipdverse.org/globalHolocene/current_version/165_1002C.lpd
downloading http://lipdverse.org/globalHolocene/current_version/18287-3.Kienast.2001.lpd
downloading http://lipdverse.org/globalHolocene/current_version/2005-804.Ledu.2013.lpd
downloading http://lipdverse.org/globalHolocene/current_version/3MPond.Pellatt.2000.lpd
downloading http://lipdverse.org/globalHolocene/current_version/74KL_TEX86.Huguet.2006.lpd
downloading http://lipdverse.org/globalHolocene/current_version/74KL_UK37.Huguet.2006.lpd
downloading http://lipdverse.org/globalHolocene/current_version/893A.Kennet.2007.lpd
downloading http://lipdverse.org/globalHolocene/current_version/A7.Sun.2005.lpd
downloading http://lipdverse.org/globalHolocene/current_version/AD9117.Guinta.2001.lpd
downloading http://lipdverse.org/globalHolocene

downloading http://lipdverse.org/globalHolocene/current_version/MD01-2444.Martrat.2007.lpd
downloading http://lipdverse.org/globalHolocene/current_version/MD01-2446.Salguero.2014.lpd
downloading http://lipdverse.org/globalHolocene/current_version/MD012378.Xu.2008.lpd
downloading http://lipdverse.org/globalHolocene/current_version/MD03_2611.lpd
downloading http://lipdverse.org/globalHolocene/current_version/MD032601.Crosta.2007.lpd
downloading http://lipdverse.org/globalHolocene/current_version/MD032707.Weldeab.2007.lpd
downloading http://lipdverse.org/globalHolocene/current_version/MD79257.lpd
downloading http://lipdverse.org/globalHolocene/current_version/MD94-103.Sicre.2005.lpd
downloading http://lipdverse.org/globalHolocene/current_version/MD952042.Bard.2002.lpd
downloading http://lipdverse.org/globalHolocene/current_version/MD952043.lpd
downloading http://lipdverse.org/globalHolocene/current_version/MD97_2151.lpd
downloading http://lipdverse.org/globalHolocene/current_version/MD97-

downloading http://lipdverse.org/globalHolocene/current_version/Agassiz.Lecavalier.2017.lpd
downloading http://lipdverse.org/globalHolocene/current_version/EDC.Jouzel.2007.lpd
downloading http://lipdverse.org/globalHolocene/current_version/PlateauRemote.MosleyThompson.1996.lpd
downloading http://lipdverse.org/globalHolocene/current_version/SIPLEMelt.Das.2008.lpd
downloading http://lipdverse.org/globalHolocene/current_version/TALDICE.Mezgec.2017.lpd
downloading http://lipdverse.org/globalHolocene/current_version/EagleLake.Rosenberg.2004.lpd
downloading http://lipdverse.org/globalHolocene/current_version/FinseStasjonsdam.Velle.2005.lpd
downloading http://lipdverse.org/globalHolocene/current_version/Kyutyunda.Pollen.Russia.lpd
downloading http://lipdverse.org/globalHolocene/current_version/LakeChalla.Climate-12k-Site-Data-Entry-Form-v3.Challa.lpd
downloading http://lipdverse.org/globalHolocene/current_version/LakeoftheWoods.Palmer.2002.lpd
downloading http://lipdverse.org/globalHolocene/c

downloading http://lipdverse.org/globalHolocene/current_version/Lofty.Lichti.1970.lpd
downloading http://lipdverse.org/globalHolocene/current_version/LakePupuke.VandenBos.2018.lpd
downloading http://lipdverse.org/globalHolocene/current_version/BoundaryPond.Mott.1977.lpd
downloading http://lipdverse.org/globalHolocene/current_version/Carp.Whitlock.1985.lpd
downloading http://lipdverse.org/globalHolocene/current_version/M_Lake.Ritchie.1977.lpd
downloading http://lipdverse.org/globalHolocene/current_version/Medicine.Radle.1992.lpd
downloading http://lipdverse.org/globalHolocene/current_version/ranger.Brubaker.1983.lpd
downloading http://lipdverse.org/globalHolocene/current_version/Rapid.Fall.1994.lpd
downloading http://lipdverse.org/globalHolocene/current_version/Koucha.Pollen.China.lpd
downloading http://lipdverse.org/globalHolocene/current_version/Colo.Baker.1990.lpd
downloading http://lipdverse.org/globalHolocene/current_version/Demont.Kapp.NA.lpd
downloading http://lipdverse.org/globa

downloading http://lipdverse.org/globalHolocene/current_version/LouisePond.Mathewes.1994.lpd
downloading http://lipdverse.org/globalHolocene/current_version/Crates.Liu.NA.lpd
downloading http://lipdverse.org/globalHolocene/current_version/PerchLake_ON.Terasmae.1975.lpd
downloading http://lipdverse.org/globalHolocene/current_version/AjaxBog.Pollen.NewZealand.lpd
downloading http://lipdverse.org/globalHolocene/current_version/Jewell.Baker.1986.lpd
downloading http://lipdverse.org/globalHolocene/current_version/GeoB5844_2.lpd
downloading http://lipdverse.org/globalHolocene/current_version/PogoniaBog.Cushing.1979.lpd
downloading http://lipdverse.org/globalHolocene/current_version/LakeBolshieToroki.Pollen.Russia.lpd
downloading http://lipdverse.org/globalHolocene/current_version/LakeEC2.Gajewski.1991.lpd
downloading http://lipdverse.org/globalHolocene/current_version/Holebudalen.Seppa.2009.lpd
downloading http://lipdverse.org/globalHolocene/current_version/waskey.Levy.2004.lpd
downloading h

downloading http://lipdverse.org/globalHolocene/current_version/Harrie.King.1986.lpd
downloading http://lipdverse.org/globalHolocene/current_version/tiavatnet.Seppa.2009.lpd
downloading http://lipdverse.org/globalHolocene/current_version/KR02.Peros.2008.lpd
downloading http://lipdverse.org/globalHolocene/current_version/k2.Fallu.2005.lpd
downloading http://lipdverse.org/globalHolocene/current_version/Reiarsdalvatnet.Seppa.2009.lpd
downloading http://lipdverse.org/globalHolocene/current_version/Gammelheimvatnet.Seppa.2009.lpd
downloading http://lipdverse.org/globalHolocene/current_version/dalene.Eide.2009.lpd
downloading http://lipdverse.org/globalHolocene/current_version/KP2.Seppa.2009.lpd
downloading http://lipdverse.org/globalHolocene/current_version/GeoB12615_4.lpd
downloading http://lipdverse.org/globalHolocene/current_version/haugtjern.Eide.2009.lpd
downloading http://lipdverse.org/globalHolocene/current_version/LakeBerkut.Ilyashuk.2005.lpd
downloading http://lipdverse.org/globalH

downloading http://lipdverse.org/globalHolocene/current_version/CottonwoodLake.Grimm.1987.lpd
downloading http://lipdverse.org/globalHolocene/current_version/hallet.McKay.2009.lpd
downloading http://lipdverse.org/globalHolocene/current_version/Corser.Nichols.2014.lpd
downloading http://lipdverse.org/globalHolocene/current_version/RiviereFeuilles2.Larouche.1981.lpd
downloading http://lipdverse.org/globalHolocene/current_version/quartz.Wooller.2012.lpd
downloading http://lipdverse.org/globalHolocene/current_version/LakeCF8.Axford.2011.lpd
downloading http://lipdverse.org/globalHolocene/current_version/JR01.Zabenskie.2007.lpd
downloading http://lipdverse.org/globalHolocene/current_version/Flarken.Seppa.2005.lpd
downloading http://lipdverse.org/globalHolocene/current_version/MD84_527.lpd
downloading http://lipdverse.org/globalHolocene/current_version/lake850.Shemesh.2001.lpd
downloading http://lipdverse.org/globalHolocene/current_version/GGC19.Farmer.2011.lpd
downloading http://lipdverse.o

In [15]:
@contextlib.contextmanager
def remember_cwd():
    """Context manager to switch back to the current working directory

    Usage::

        with remember_cwd():
            os.chdir('test')
            print(os.getcwd())  # test
        print(os.getcwd())      # test/.."""
    curdir = os.getcwd()
    try:
        yield
    except:
        raise
    finally:
        os.chdir(curdir)

In [28]:
%%time
with remember_cwd():
    os.chdir('data/')
    data = lipd.readLipd('.')

    data = {key: data[key] for key in set(np.unique(df.index)).intersection(data)}

Disclaimer: LiPD files may be updated and modified to adhere to standards

Found: 618 LiPD file(s)
reading: Hams.Bennett.1987.lpd
reading: EagleTarn.Rees.2010.lpd
reading: HjortSo.Wagner.2008.lpd
reading: Alley.GISP2.2000.lpd
reading: Victoria.Berke.2012.lpd
reading: Frozen.Rosenberg.2004.lpd
reading: GeoB9307_3.lpd
reading: Composit_MD012421,KR0206.lpd
reading: FloatingIsland.Baker.1976.lpd
reading: LakeSix.Liu.1990.lpd
reading: ScreamingLynxLake.Clegg.2011.lpd
reading: Malawi.Johnson.2016.lpd
reading: WCA3B_GumboLimboFarTail.Willard.2001.lpd
reading: Coghill.King.1986.lpd
reading: LacduSommet.Hausmann.2011.lpd
reading: BanksIsland74MS15.Gajewski.2000.lpd
reading: Sagistalsee.Pollen.Switzerland.lpd
reading: MD03_2611.lpd
reading: ODP1084B.lpd
reading: GiK18515_3.lpd
reading: DolgoeOzero.Pollen.Russia.lpd
reading: MD06_3075.lpd
reading: gunnarsfjorden.Allen.2007.lpd
reading: MundayCreek.Peteet.1986.lpd
reading: sjuuodjijaure.Rosen.2001.lpd
reading: Lochnagar.Dalton.2005.lpd
reading: la

reading: M75_3_137_3.lpd
reading: NorthCraterLake.Palmer.2002.lpd
reading: BloodPond.Marsicek.2013T.lpd
reading: s53s52.Tillman.2010.lpd
reading: farewell.Hu.1998.lpd
reading: nautajarvi.Ojala.2009.lpd
reading: MD99-2317.Jennings.2011.lpd
reading: SN-6.STiwari.2015.lpd
reading: XingyunLake.Wu.2018.lpd
reading: GYXO.Self.2015.lpd
reading: Porphyry.Brown.2002.lpd
reading: hudson.Clegg.2011.lpd
reading: Mount_Honey.lpd
reading: tibetanus.Hammarlund.2002.lpd
reading: FinseStasjonsdam.Velle.2005.lpd
reading: Devils.Maher.1982.lpd
reading: Kharinei.Pollen.Russia.lpd
reading: 893A.Kennet.2007.lpd
reading: LeadingTickles.Macpherson.1985.lpd
reading: SouthernCalifornia.OhlweinWahl.2012.lpd
reading: starvatn.Andresen.2006.lpd
reading: Crates.Liu.NA.lpd
reading: ParkPond1.Lynch.1998.lpd
reading: PeatlandKlukva.Pollen.Russia.lpd
reading: Compass.Macpherson.1986.lpd
reading: Mansell.Shuman.2016.lpd
reading: GeoB65181.Schefu.2005.lpd
reading: MD07-3104.Pollen.Chile.lpd
reading: TangledUpLake.Anderso

Extract all the paleoData series from the data

In [29]:
%%time
all_series = lipd.extractTs(data)

extracting paleoData...
extracting: JM96-1207.Solignac.2013
extracting: Medicine.Radle.1992
extracting: FloatingIsland.Baker.1976
extracting: ParkPond3.Lynch.1998
extracting: JamesRossIsland.Mulvaney.2012
extracting: Irvin.Cushing.1982
extracting: Corser.Nichols.2014
extracting: Windy.Chase.2008
extracting: trehorningen.Antonsson.2007
extracting: MD99-2322.Jennings.2011
extracting: MooseLake.Clegg.2010
extracting: Decoy.Szeicz.1991
extracting: GiK18515_3
extracting: tibetanus.Hammarlund.2002
extracting: LoonPond.Jacobson.Ja
extracting: austerkjosen.Seppa.2009
extracting: sfl4-1.Willemse.1999
extracting: Harrie.King.1986
extracting: Sud_duLac_duNoyer.Larouche.1977
extracting: Nina.Liu.1990
extracting: Reidel.Almquist.1992
extracting: LeadingTickles.Macpherson.1985
extracting: HU90.deVernal.2013
extracting: MD03_2607
extracting: CarletonLake(informalname).Upiter.2014
extracting: NorthPond.Whitehead.1978
extracting: BanksIsland74MS11.Gajewski.2000
extracting: SP10WINZ.Williams.2010
extrac

extracting: LakeBolshieToroki.Pollen.Russia
extracting: Bastien.Bennett.1987
extracting: tornetrask.Grudd.2002
extracting: HU91.Levac.2013
extracting: candelabra.Cwynar.1995
extracting: gunnarsfjorden.Allen.2007
extracting: LacColin.Mott.1977
extracting: LakeSix.Liu.1990
extracting: P1003.Sejrup.2011
extracting: EDML.Stenni.2009
extracting: CottonwoodPassPond.Fall.1997
extracting: GEOFAR_KF16_MgCa.Repschlager.2016
extracting: Kotokel.Tarasov.2009
extracting: MD952042.Bard.2002
extracting: Ferndale.Albert.1981
extracting: Iglutalik.Davis.NA
extracting: hallet.McKay.2009
extracting: Brandreth.Overpeck.1985
extracting: LacMartyne.McAndrews.NA
extracting: HeihaiLake.Chang.2017
extracting: HM03_133_25
extracting: PerchLake_Manitoba.Ritchie.NA
extracting: Clearwater.Mott.1973
extracting: Abalone.Cole.1994
extracting: MSM055_712-2.Werner.2015
extracting: LawDome.Dahl-Jensen.1999
extracting: SchwarzseeobSoelden.Ilyashuk.2011
extracting: PolevaCave.Constantin.2007
extracting: M39_008
extracting

Extract only the temperature series with units in degrees Celsius

In [45]:
filtered_ts_temp12k = lipd.filterTs(all_series,'paleoData_inCompilation == Temp12k')
filtered_ts_useinglobal = lipd.filterTs(filtered_ts_temp12k,'paleoData_useInGlobalTemperatureAnalysis == TRUE')
temperatures = lipd.filterTs(filtered_ts_useinglobal,'paleoData_units == degC')
sorted(temperatures[0])

Found 647 matches from 2369 columns
Found 363 matches from 647 columns
Found 335 matches from 363 columns


['@context',
 'age',
 'ageUnits',
 'agesPerKyr',
 'archiveType',
 'createdBy',
 'dataSetName',
 'geo_AHT',
 'geo_countryOcean',
 'geo_meanElev',
 'geo_meanLat',
 'geo_meanLon',
 'geo_siteName',
 'geo_type',
 'lipdVersion',
 'lipdverseLink',
 'maxYear',
 'minYear',
 'mode',
 'nUniqueAges',
 'notes',
 'originalDataURL',
 'originalDataUrl',
 'paleoData_QCCertification',
 'paleoData_QCLastUpdated',
 'paleoData_TSid',
 'paleoData_datum',
 'paleoData_filename',
 'paleoData_hasMaxValue',
 'paleoData_hasMeanValue',
 'paleoData_hasMedianValue',
 'paleoData_hasMinValue',
 'paleoData_hasResolution_hasMaxValue',
 'paleoData_hasResolution_hasMeanValue',
 'paleoData_hasResolution_hasMedianValue',
 'paleoData_hasResolution_hasMinValue',
 'paleoData_inCompilation',
 'paleoData_interpretation',
 'paleoData_meanValue12k',
 'paleoData_measurementTableMD5',
 'paleoData_measurementTableName',
 'paleoData_medianRes12k',
 'paleoData_missingValue',
 'paleoData_number',
 'paleoData_proxy',
 'paleoData_proxyDet

<a id='meta_cols'></a>Transform the temperatures into pandas series. The `meta_cols` is the meta data that should be available as column in the data frame and has to match one of the keys in the previous list. The `meta_names` then specifies how the corresponding field from `meta_cols` appears in the final data frame.

In [46]:
meta_cols = ['geo_meanLon', 'geo_meanLat', 'dataSetName', 'paleoData_variableName']
meta_names = ['lon', 'lat', 'dataSetName', 'variableName']

series = [
    pd.Series(
        np.asarray(d['paleoData_values'], dtype=float),
        index=np.asarray(d['age'], dtype=float),
        name=tuple(d.get(name, np.nan) for name in meta_cols))
    for d in temperatures if 'age' in d]

and bin them based on centennial scales and merge them into one single dataframe

In [47]:
def age_grouper(age):
    """Bin age to centuries"""
    return age - (age % binwidth)

In [48]:
binned = [s.groupby(age_grouper).mean() for s in series]

merged = binned[0].to_frame()
merged.columns.names = meta_names

for s in binned[1:]:
    merged = merged.merge(s.to_frame(), left_index=True, right_index=True,
                          how='outer')
    
merged = merged.T

Now save the results

In [49]:
merged.to_csv('data/binned-temperature-data.tsv', '\t')

df.loc[np.unique(merged.index.get_level_values('dataSetName'))].to_csv('data/meta.tsv', '\t')

<a id='final'></a>That's it! If the notebook has finished, you can download the temperature data [here](data/binned-temperature-data.tsv) and the corresponding meta data [here](data/meta.tsv) as tab-separated files.

So let's have a look into the final temperature data:

In [50]:
merged

,,,,-200.0,-100.0,0.0,100.0,200.0,300.0,400.0,500.0,600.0,700.0,...,1237500.0,1240500.0,1241800.0,1243100.0,1261400.0,1263300.0,1264800.0,1266200.0,1267900.0,1277300.0
lon,lat,dataSetName,variableName,,,,,,,,,,,,,,,,,,,,,
-29.35,68.1000,JM96-1207.Solignac.2013,temperature,NaN,NaN,NaN,NaN,NaN,NaN,5.884000,3.462000,2.112000,4.8930,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-97.35,44.9800,Medicine.Radle.1992,temperature,NaN,NaN,4.538857,4.493500,4.50700,4.70700,4.283000,4.637000,4.430000,4.4100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-107.47,44.5500,FloatingIsland.Baker.1976,temperature,NaN,NaN,2.167000,NaN,NaN,NaN,NaN,NaN,2.808000,2.2620,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-109.92,43.4600,ParkPond3.Lynch.1998,temperature,NaN,NaN,NaN,NaN,NaN,NaN,1.185000,NaN,NaN,0.7150,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-57.685,-64.2017,JamesRossIsland.Mulvaney.2012,temperature,NaN,0.242000,-0.299000,-0.011000,-0.27400,-0.40500,-0.685000,-0.987000,-0.351000,-0.8690,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-93.64,47.1400,Irvin.Cushing.1982,temperature,NaN,5.092000,NaN,NaN,4.64400,4.44200,3.979000,NaN,3.979000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-145.45,60.5300,Corser.Nichols.2014,temperature,NaN,NaN,NaN,4.870000,NaN,5.06000,NaN,5.280000,NaN,6.7560,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-117.8772,49.8133,Windy.Chase.2008,temperature,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.260000,9.070000,9.1700,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11.6,58.5500,trehorningen.Antonsson.2007,temperature,NaN,NaN,6.354000,NaN,8.20600,NaN,NaN,NaN,NaN,7.3130,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
merged.loc[merged.iloc[:, 0].notnull()]

,,,,-200.0,-100.0,0.0,100.0,200.0,300.0,400.0,500.0,600.0,700.0,...,1237500.0,1240500.0,1241800.0,1243100.0,1261400.0,1263300.0,1264800.0,1266200.0,1267900.0,1277300.0
lon,lat,dataSetName,variableName,,,,,,,,,,,,,,,,,,,,,
-73.05,42.65,NorthPond.Whitehead.1978,temperature,6.139,NaN,NaN,NaN,NaN,NaN,NaN,6.511,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-99.91,44.84,CottonwoodLake.Grimm.1987,temperature,5.274,6.617,7.012,5.073,6.677,NaN,5.729,4.793,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
